In [0]:
from pyspark.sql.functions import col
from delta.tables import DeltaTable
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),        # Log to console
        logging.FileHandler('silver_layer.log')  # log to a file 
    ]
)
logger = logging.getLogger(__name__)

# Set catalog and schema names
bronze_catalog = "bronze"
bronze_schema = "training_io"
silver_catalog = "silver"
silver_schema = "training_io"

# Set table names
bronze_teams = f"{bronze_catalog}.{bronze_schema}.teams"
bronze_players = f"{bronze_catalog}.{bronze_schema}.players"
bronze_goals = f"{bronze_catalog}.{bronze_schema}.goals_data"

silver_teams = f"{silver_catalog}.{silver_schema}.teams"
silver_players = f"{silver_catalog}.{silver_schema}.players"
silver_goals = f"{silver_catalog}.{silver_schema}.goals"

def upsert(source_df, silver_table, key_columns, timestamp_column="load_ts"):
    """
    Perform SCD1 upsert (merge) into a Delta table using load_ts timestamp comparison.

    Parameters:
    - source_df:        Source Spark DataFrame
    - silver_table:     Target Delta table name
    - key_columns:      List of key columns to match records
    - timestamp_column: Name of the timestamp field used for freshness comparison (default: 'load_ts').
    """
    try:
        # Build the join condition for merge
        join_condition = " AND ".join([
            f"target.{col} = source.{col}" for col in key_columns
        ])

        # Set values to update from source
        update_set = {col: f"source.{col}" for col in source_df.columns}

        if spark.catalog.tableExists(silver_table):
            delta_table = DeltaTable.forName(spark, silver_table)

            logger.info(f"Merging into existing table: {silver_table}")

            delta_table.alias("target").merge(
                source_df.alias("source"),
                join_condition
            ).whenMatchedUpdate(
                condition=f"source.{timestamp_column} > target.{timestamp_column}",
                set=update_set
            ).whenNotMatchedInsertAll().execute()

            logger.info(f"Upsert for table {silver_table} is completed!")

        else:
            logger.info(f"Creating new table: {silver_table}")
            source_df.write.format("delta").mode("overwrite").saveAsTable(silver_table)

    except Exception as e:
        logger.error(f"Error during upsert operation for table {silver_table}: {e}")

def overwrite(source_df, silver_table):
    """
    Overwrite the entire Delta table with new data.
    
    Parameters:
    - source_df: Source DataFrame to overwrite the table
    - silver_table: Target Delta table name to be overwritten
    """
    try:
        source_df.write.format("delta").mode("overwrite").saveAsTable(silver_table)
        logger.info(f"Overwrite for table {silver_table} is completed!")
    except Exception as e:
        logger.error(f"Error during overwrite operation for table {silver_table}: {e}")

def main():
    try:
        # Read data from bronze layer
        bronze_teams_df = spark.read.table(bronze_teams)
        bronze_players_df = spark.read.table(bronze_players)
        bronze_goals_df = spark.read.table(bronze_goals)

        # Call the upsert function for teams
        upsert(bronze_teams_df, silver_teams, ["id_team"])

        bronze_players_df = bronze_players_df.select("id_player",
            "player_name",
            "nationality",
            "field_position",
            "id_team",
            "player_image",
            "bio.age",
            "bio.height",
            "bio.weight",
            "load_ts"
        )

        upsert(bronze_players_df, silver_players, ["id_player"])

        overwrite(bronze_goals_df, silver_goals)

    except Exception as e:
        logger.error(f"Error in main function: {e}")

# Run main
main()

In [0]:
# # If the merge executed, check the result of the operation
# delta_table = DeltaTable.forName(spark, silver_teams)
# history_df = delta_table.history(1)  # Get the most recent operation

# # Get the operation metrics for the most recent change
# operation_metrics = history_df.select("operationMetrics").collect()[0][0]
# print(operation_metrics)
